# CMPE 181 Homework 1 
Danielle Shen 

CMPE 181 

Prof Kaikai Liu 

Due: March 21, 2021


# Install required packages 
Some of them are already installed in Colab

In [ ]:
!pip install google-cloud-iot

In [ ]:
!pip install google-api-python-client

In [ ]:
!pip install google-cloud-storage

In [ ]:
!pip install cryptography pyjwt paho-mqtt 

     |████████████████████████████████| 3.2MB 5.6MB/s 
     |████████████████████████████████| 102kB 7.6MB/s 
  Created wheel for paho-mqtt: filename=paho_mqtt-1.5.1-cp37-none-any.whl size=61546 sha256=67226528ddcc54efb0e79421c69c1fc01775abf00461c1258d6142f29642193f
  Stored in directory: /root/.cache/pip/wheels/75/e2/f5/78942b19b4d135605e58dfe85fba52253b14d636aabf76904b
Successfully built paho-mqtt


In [ ]:
import paho.mqtt.client as mqtt
import jwt
import time
import ssl
import random
import os
import logging
import datetime
import argparse

# Setup Google Cloud Client

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config list

[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
[core]
account = danielle.shen@sjsu.edu

Your active configuration is: [default]


In [ ]:
!gcloud config list project

[core]
project (unset)

Your active configuration is: [default]


In [ ]:
!gcloud config set project 'cmpe181hw-308102'

Updated property [core/project].


In [ ]:
!gcloud config list project

[core]
project = cmpe181hw-308102

Your active configuration is: [default]


In [ ]:
!gcloud config set compute/region us-central1

Updated property [compute/region].
API [compute.googleapis.com] not enabled on project [598667626269]. 
Would you like to enable and retry (this will take a few minutes)? 
(y/N)?  y

Enabling service [compute.googleapis.com] on project [598667626269]...
Operation "operations/acf.p2-598667626269-fb502c65-4479-4b5d-83bd-fe71f805f6cf" finished successfully.


In [ ]:
!gcloud compute zones list

NAME                       REGION                   STATUS  NEXT_MAINTENANCE  TURNDOWN_DATE
us-east1-b                 us-east1                 UP
us-east1-c                 us-east1                 UP
us-east1-d                 us-east1                 UP
us-east4-c                 us-east4                 UP
us-east4-b                 us-east4                 UP
us-east4-a                 us-east4                 UP
us-central1-c              us-central1              UP
us-central1-a              us-central1              UP
us-central1-f              us-central1              UP
us-central1-b              us-central1              UP
us-west1-b                 us-west1                 UP
us-west1-c                 us-west1                 UP
us-west1-a                 us-west1                 UP
europe-west4-a             europe-west4             UP
europe-west4-b             europe-west4             UP
europe-west4-c             europe-west4             UP
europe-west1-b             e

In [197]:
!gcloud config set compute/zone us-west2-a

Updated property [compute/zone].


# Mount Google Drive

Lecture 10 - 42:56

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Create a service credentials, ref: https://cloud.google.com/storage/docs/reference/libraries#client-libraries-usage-nodejs

In [54]:
myservicecredentials='/content/gdrive/"My Drive"/"Colab Notebooks"/Googlecerts'

In [58]:
!ls {myservicecredentials}

CMPE181HW-45a9908167ee.json  iotdevicenew


In [59]:
newdevicecredentials_path=os.path.join(myservicecredentials, 'iotdevicenew')
!ls {newdevicecredentials_path}

rsa_cert.pm  rsa_private.pem


In [61]:
myservicecredentials_path=os.path.join(myservicecredentials, 'CMPE181HW-45a9908167ee.json')
#Service account credentials

In [62]:
!cp {myservicecredentials_path} . #copy to the local directory

In [63]:
!cp -r {newdevicecredentials_path} .

In [64]:
!ls

adc.json  CMPE181HW-45a9908167ee.json  gdrive  iotdevicenew  sample_data


In [68]:
rootcredentials_path=os.path.join(myservicecredentials, 'roots.pem')
!cp {rootcredentials_path} .

You can retrieve the Google root certificate. For example, wget https://pki.goog/roots.pem or curl https://pki.goog/roots.pem > roots.pem

In [69]:
!ls

adc.json		     gdrive	   roots.pem
CMPE181HW-45a9908167ee.json  iotdevicenew  sample_data


In [70]:
GOOGLE_APPLICATION_CREDENTIALS='./CMPE181HW-45a9908167ee.json'

In [71]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

./CMPE181HW-45a9908167ee.json


# Google Cloud Storage Gsutil

Lecture 10 - 46:14

In [73]:
from google.cloud import storage

In [80]:
# Instantiates a client
storage_client = storage.Client.from_service_account_json('./CMPE181HW-45a9908167ee.json')
#storage_client = storage.Client()

In [81]:
!gsutil ls #list google cloud storage bucket

gs://cmpe181_image/


In [ ]:
# The name for the new bucket
bucket_name = "cmpe181_image"
# Creates the new bucket
bucket = storage_client.create_bucket(bucket_name)
print("Bucket {} created.".format(bucket.name))

In [ ]:
blob = bucket.blob('sjsu/img1.jpg') #(folder, filename)
blob.upload_from_filename('./sjsu-virtual-open-house-social_v2.jpg')

In [ ]:
#upload image to an existing bucket
bucketexist = storage_client.bucket('cmpelkk_imagetest')
blobexist = bucketexist.blob('sjsu/img1.jpg')
blobexist.upload_from_filename('./sjsu-virtual-open-house-social_v2.jpg')

# Google IoT Defined functions

In [148]:
import paho.mqtt.client as mqtt
import jwt
import time
import ssl
import random
import os
import logging
import datetime
import argparse

from google.cloud import storage

In [149]:
# The initial backoff time after a disconnection occurs, in seconds.
minimum_backoff_time = 1

# The maximum backoff time before giving up, in seconds.
MAXIMUM_BACKOFF_TIME = 32

# Whether to wait with exponential backoff before publishing.
should_backoff = False

def create_jwt(project_id, private_key_file, algorithm):
    """Creates a JWT (https://jwt.io) to establish an MQTT connection.
        Args:
         project_id: The cloud project ID this device belongs to
         private_key_file: A path to a file containing either an RSA256 or
                 ES256 private key.
         algorithm: The encryption algorithm to use. Either 'RS256' or 'ES256'
        Returns:
            A JWT generated from the given project_id and private key, which
            expires in 20 minutes. After 20 minutes, your client will be
            disconnected, and a new JWT will have to be generated.
        Raises:
            ValueError: If the private_key_file does not contain a known key.
        """

    token = {
        # The time that the token was issued at
        'iat': datetime.datetime.utcnow(),
        # The time the token expires.
        'exp': datetime.datetime.utcnow() + datetime.timedelta(minutes=20),
        # The audience field should always be set to the GCP project id.
        'aud': project_id
    }

    # Read the private key file.
    with open(private_key_file, 'r') as f:
        private_key = f.read()

    print('Creating JWT using {} from private key file {}'.format(
        algorithm, private_key_file))

    return jwt.encode(token, private_key, algorithm=algorithm)
# [END iot_mqtt_jwt]

In [150]:
def error_str(rc):
    """Convert a Paho error to a human readable string."""
    return '{}: {}'.format(rc, mqtt.error_string(rc))


def on_connect(unused_client, unused_userdata, unused_flags, rc):
    """Callback for when a device connects."""
    print('on_connect', mqtt.connack_string(rc))

    # After a successful connect, reset backoff time and stop backing off.
    global should_backoff
    global minimum_backoff_time
    should_backoff = False
    minimum_backoff_time = 1


def on_disconnect(unused_client, unused_userdata, rc):
    """Paho callback for when a device disconnects."""
    print('on_disconnect', error_str(rc))

    # Since a disconnect occurred, the next loop iteration will wait with
    # exponential backoff.
    global should_backoff
    should_backoff = True


def on_publish(unused_client, unused_userdata, unused_mid):
    """Paho callback when a message is sent to the broker."""
    print('on_publish')


def on_message(unused_client, unused_userdata, message):
    """Callback when the device receives a message on a subscription."""
    payload = str(message.payload.decode('utf-8'))
    print('Received message \'{}\' on topic \'{}\' with Qos {}'.format(
        payload, message.topic, str(message.qos)))

Lecture 10 - 52:30

In [193]:
def get_client(
        project_id, cloud_region, registry_id, device_id, private_key_file,
        algorithm, ca_certs, mqtt_bridge_hostname, mqtt_bridge_port):
    """Create our MQTT client. The client_id is a unique string that identifies
    this device. For Google Cloud IoT Core, it must be in the format below."""
    client_id = 'projects/{}/locations/{}/registries/{}/devices/{}'.format(
        project_id, cloud_region, registry_id, device_id)
    print('Device client_id is \'{}\''.format(client_id))

    client = mqtt.Client(client_id=client_id)

    # With Google Cloud IoT Core, the username field is ignored, and the
    # password field is used to transmit a JWT to authorize the device.
    client.username_pw_set(
        username='unused',
        password=create_jwt(
            project_id, private_key_file, algorithm))

    # Enable SSL/TLS support.
    client.tls_set(ca_certs=ca_certs, tls_version=ssl.PROTOCOL_TLSv1_2)

    # Register message callbacks. https://eclipse.org/paho/clients/python/docs/
    # describes additional callbacks that Paho supports. In this example, the
    # callbacks just print to standard out.
    client.on_connect = on_connect
    client.on_publish = on_publish
    client.on_disconnect = on_disconnect
    client.on_message = on_message

    # Connect to the Google MQTT bridge.
    client.connect(mqtt_bridge_hostname, mqtt_bridge_port)

    # This is the topic that the device will receive configuration updates on.
    mqtt_config_topic = '/devices/{}/config'.format(device_id)

    # Subscribe to the config topic.
    client.subscribe(mqtt_config_topic, qos=1)

    # The topic that the device will receive commands on.
    mqtt_command_topic = '/devices/{}/commands/#'.format(device_id)

    # Subscribe to the commands topic, QoS 1 enables message acknowledgement.
    print('Subscribing to {}'.format(mqtt_command_topic))
    client.subscribe(mqtt_command_topic, qos=0)

    return client
# [END iot_mqtt_config]

In [194]:
!ls

adc.json		     gdrive	   roots.pem
CMPE181HW-45a9908167ee.json  iotdevicenew  sample_data


Lecture 10 - 56:00

In [196]:
GOOGLE_APPLICATION_CREDENTIALS='./CMPE181HW-45a9908167ee.json'
ROOT_CREDENTIALS='./roots.pem'

edit: device id
registry id 
json

possible path 


'/content/gdrive/"My Drive"/"Colab Notebooks"/"Googlecerts"/"iotdevicenew"/rsa_private.pem'

In [198]:
class Args:
  algorithm = 'RS256'
  ca_certs = '/content/roots.pem'
  cloud_region = 'us-central1'
  data = 'Hello there'
  device_id = 'cmpe181dev1'
  jwt_expires_minutes = 20
  listen_dur = 60
  message_type = 'event'
  mqtt_bridge_hostname = 'mqtt.googleapis.com'
  mqtt_bridge_port = 8883
  num_messages = 20
  private_key_file = '/content/iotdevicenew/rsa_private.pem'
  #/content/gdrive/"My Drive"/CurrentWork/CMPE181Sp2020/Googlecerts/cmpe181dev1/rsa_private.pem'
  project_id = 'cmpe181hw-308102'
  registry_id = 'CMPEIoT1'
  service_account_json = '/content/CMPE181HW-45a9908167ee.json'#os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")
  imagefolder_path= './data'


args=Args()

In [199]:
print(args.private_key_file)

/content/iotdevicenew/rsa_private.pem


In [200]:

def parse_command_line_args():
    """Parse command line arguments."""
    parser = argparse.ArgumentParser(description=(
        'Example Google Cloud IoT Core MQTT device connection code.'))
    parser.add_argument(
        '--algorithm',
        choices=('RS256', 'ES256'),
        required=True,
        help='Which encryption algorithm to use to generate the JWT.')
    parser.add_argument(
        '--ca_certs',
        default='./roots.pem',
        help='CA root from https://pki.google.com/roots.pem')
    parser.add_argument(
        '--cloud_region', default='us-central1', help='GCP cloud region')
    parser.add_argument(
        '--data',
        default='Hello there',
        help='The telemetry data sent on behalf of a device')
    parser.add_argument(
        '--device_id', required=True, help='Cloud IoT Core device id')
    parser.add_argument(
        '--gateway_id', required=False, help='Gateway identifier.')
    parser.add_argument(
        '--jwt_expires_minutes',
        default=20,
        type=int,
        help='Expiration time, in minutes, for JWT tokens.')
    parser.add_argument(
        '--listen_dur',
        default=60,
        type=int,
        help='Duration (seconds) to listen for configuration messages')
    parser.add_argument(
        '--message_type',
        choices=('event', 'state'),
        default='event',
        help=('Indicates whether the message to be published is a '
              'telemetry event or a device state message.'))
    parser.add_argument(
        '--mqtt_bridge_hostname',
        default='mqtt.googleapis.com',
        help='MQTT bridge hostname.')
    parser.add_argument(
        '--mqtt_bridge_port',
        choices=(8883, 443),
        default=8883,
        type=int,
        help='MQTT bridge port.')
    parser.add_argument(
        '--num_messages',
        type=int,
        default=100,
        help='Number of messages to publish.')
    parser.add_argument(
        '--private_key_file',
        required=True,
        help='Path to private key file.')
    parser.add_argument(
        '--project_id',
        default=os.environ.get('GOOGLE_CLOUD_PROJECT'),
        help='GCP cloud project name')
    parser.add_argument(
        '--registry_id', required=True, help='Cloud IoT Core registry id')
    parser.add_argument(
        '--service_account_json',
        default=os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"),
        help='Path to service account json file.')

    return parser.parse_args()


def mqtt_device_demo(args):
    """Connects a device, sends data, and receives data."""
    # [START iot_mqtt_run]
    global minimum_backoff_time
    global MAXIMUM_BACKOFF_TIME

    # Publish to the events or state topic based on the flag.
    sub_topic = 'events' if args.message_type == 'event' else 'state'

    mqtt_topic = '/devices/{}/{}'.format(args.device_id, sub_topic)

    jwt_iat = datetime.datetime.utcnow()
    jwt_exp_mins = args.jwt_expires_minutes
    client = get_client(
        args.project_id, args.cloud_region, args.registry_id,
        args.device_id, args.private_key_file, args.algorithm,
        args.ca_certs, args.mqtt_bridge_hostname, args.mqtt_bridge_port)

    # Publish num_messages messages to the MQTT bridge once per second.
    for i in range(1, args.num_messages + 1):
        # Process network events.
        client.loop()

        # Wait if backoff is required.
        if should_backoff:
            # If backoff time is too large, give up.
            if minimum_backoff_time > MAXIMUM_BACKOFF_TIME:
                print('Exceeded maximum backoff time. Giving up.')
                break

            # Otherwise, wait and connect again.
            delay = minimum_backoff_time + random.randint(0, 1000) / 1000.0
            print('Waiting for {} before reconnecting.'.format(delay))
            time.sleep(delay)
            minimum_backoff_time *= 2
            client.connect(args.mqtt_bridge_hostname, args.mqtt_bridge_port)

        payload = '{}/{}-payload-{}'.format(
            args.registry_id, args.device_id, i)
        print('Publishing message {}/{}: \'{}\''.format(
            i, args.num_messages, payload))
        # [START iot_mqtt_jwt_refresh]
        seconds_since_issue = (datetime.datetime.utcnow() - jwt_iat).seconds
        if seconds_since_issue > 60 * jwt_exp_mins:
            print('Refreshing token after {}s'.format(seconds_since_issue))
            jwt_iat = datetime.datetime.utcnow()
            client.loop()
            client.disconnect()
            client = get_client(
                args.project_id, args.cloud_region,
                args.registry_id, args.device_id, args.private_key_file,
                args.algorithm, args.ca_certs, args.mqtt_bridge_hostname,
                args.mqtt_bridge_port)
        # [END iot_mqtt_jwt_refresh]
        # Publish "payload" to the MQTT topic. qos=1 means at least once
        # delivery. Cloud IoT Core also supports qos=0 for at most once
        # delivery.
        client.publish(mqtt_topic, payload, qos=1)

        
        # Send events every second. State should not be updated as often
        time.sleep(1)
        # for i in range(0, 60):
        #     time.sleep(1)
        #     client.loop()
    # [END iot_mqtt_run]

Error: can't find the private key

In [201]:
mqtt_device_demo(args)

Device client_id is 'projects/cmpe181hw-308102/locations/us-central1/registries/CMPEIoT1/devices/cmpe181dev1'
Creating JWT using RS256 from private key file /content/iotdevicenew/rsa_private.pem
Subscribing to /devices/cmpe181dev1/commands/#
Exceeded maximum backoff time. Giving up.



```Device client_id is 'projects/cmpe181hw-308102/locations/us-west2-a/registries/CMPEIoT1/devices/cmpe181dev1'
Creating JWT using RS256 from private key file /content/iotdevicenew/rsa_private.pem
Subscribing to /devices/cmpe181dev1/commands/#
Exceeded maximum backoff time. Giving up. 




getting a weird result for mqtt_device_demo(agrs) command 

"Exceeded maximum backoff time. Giving up."

^^ error occurs when path is /content/filename

Device client_id is 'projects/	cmpe181hw-308102/locations/us-central1/registries/CMPEIoT1/devices/cmpeiotdevice2'
Creating JWT using RS256 from private key file /content/iotdevicenew/rsa_private.pem
Subscribing to /devices/cmpeiotdevice2/commands/#
Publishing message 1/20: 'CMPEIoT1/cmpeiotdevice2-payload-1'
on_disconnect 1: Out of memory.


In [129]:
import random
import time
import datetime
import json

def read_sensor(count):
    tempF = 20 + 0.2*count + (random.random() * 15)
    humidity = 60 + 0.3*count+ (random.random() * 20)
    temp = '{0:0.2f}'.format(tempF)
    hum = '{0:0.2f}'.format(humidity)
    sensorZipCode = 95192#"94043"
    sensorLat = 37.3382082+ (random.random() /100)#"37.421655"
    sensorLong = -121.8863286 + (random.random() /100)#"-122.085637"
    sensorLatf = '{0:0.6f}'.format(sensorLat)
    sensorLongf = '{0:0.6f}'.format(sensorLong)
    return (temp, hum, sensorZipCode, sensorLatf, sensorLongf)

def createJSON(reg_id, dev_id, timestamp, zip, lat, long, temperature, humidity):
    data = {
      'registry_id' : reg_id,
      'device_id' : dev_id,
      'timecollected' : timestamp,
      'zipcode' : zip,
      'latitude' : lat,
      'longitude' : long,
      'temperature' : temperature,
      'humidity' : humidity
    }

    json_str = json.dumps(data)
    return json_str

# 39:20 in lecture 10

def simulatesensor_mqtt_device_demo(args):

    """Connects a device, sends data, and receives data."""
    # [START iot_mqtt_run]
    global minimum_backoff_time
    global MAXIMUM_BACKOFF_TIME

    # Publish to the events or state topic based on the flag.
    sub_topic = 'events' if args.message_type == 'event' else 'state'

    mqtt_topic = '/devices/{}/{}'.format(args.device_id, sub_topic)

    jwt_iat = datetime.datetime.utcnow()
    jwt_exp_mins = args.jwt_expires_minutes
    client = get_client(
        args.project_id, args.cloud_region, args.registry_id,
        args.device_id, args.private_key_file, args.algorithm,
        args.ca_certs, args.mqtt_bridge_hostname, args.mqtt_bridge_port)

    # Publish num_messages messages to the MQTT bridge once per second.
    for i in range(1, args.num_messages + 1):
        client.loop()

        currentTime = datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        (temp, hum, sensorZipCode, sensorLat, sensorLong) = read_sensor(i)
        #(id, timestamp, zip, lat, long, temperature, humidity, img_file)
        payloadJSON = createJSON(args.registry_id, args.device_id, currentTime, sensorZipCode, sensorLat, sensorLong, temp, hum)

        #payload = '{}/{}-image-{}'.format(args.registry_id, args.device_id, i)
        print('Publishing message {}/: \'{}\''.format(
            i, payloadJSON))

        # Publish "payload" to the MQTT topic. qos=1 means at least once
        # delivery. Cloud IoT Core also supports qos=0 for at most once
        # delivery.
        client.publish(mqtt_topic, payloadJSON, qos=1)

        
        # Send events every second. State should not be updated as often
        time.sleep(1)
    
    

In [130]:
simulatesensor_mqtt_device_demo(args)

Device client_id is 'projects/	cmpe181hw-308102/locations/us-central1/registries/CMPEIoT1/devices/cmpeiotdevice2'
Creating JWT using RS256 from private key file /content/iotdevicenew/rsa_private.pem
Subscribing to /devices/cmpeiotdevice2/commands/#
Publishing message 1/: '{"registry_id": "CMPEIoT1", "device_id": "cmpeiotdevice2", "timecollected": "2021-03-19 05:50:31", "zipcode": 95192, "latitude": "37.346107", "longitude": "-121.883030", "temperature": "20.44", "humidity": "80.29"}'
on_disconnect 1: Out of memory.
Publishing message 2/: '{"registry_id": "CMPEIoT1", "device_id": "cmpeiotdevice2", "timecollected": "2021-03-19 05:50:32", "zipcode": 95192, "latitude": "37.345481", "longitude": "-121.886204", "temperature": "24.64", "humidity": "69.51"}'
Publishing message 3/: '{"registry_id": "CMPEIoT1", "device_id": "cmpeiotdevice2", "timecollected": "2021-03-19 05:50:33", "zipcode": 95192, "latitude": "37.338492", "longitude": "-121.881567", "temperature": "28.28", "humidity": "68.82"}'

In [135]:
!gcloud pubsub topics list

---
name: projects/cmpe181hw-308102/topics/cmpeiotdevice1


In [134]:
!gcloud pubsub subscriptions list

---
ackDeadlineSeconds: 10
expirationPolicy:
  ttl: 2678400s
messageRetentionDuration: 604800s
name: projects/cmpe181hw-308102/subscriptions/iot
pushConfig: {}
topic: projects/cmpe181hw-308102/topics/cmpeiotdevice1
